In [ ]:
import torch
import coremltools as ct
import numpy as np
import PIL
import matplotlib.pyplot as plt
import json

In [ ]:
traced_model_path = "/path/to/DeepLabV3Plus-mobileone_s3.pt"

traced_model = torch.jit.load(traced_model_path)

traced_model

In [ ]:
input_width = 800
input_height = 800

# {'input_space': 'RGB', 'input_range': [0, 1], 'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}
std = [0.229, 0.224, 0.225]
mean = [0.485, 0.456, 0.406]
scale = 1 / (np.mean(std) * 255.0)
bias = [
    - mean[0] / std[0],
    - mean[1] / std[1],
    - mean[2] / std[2],
]

input_shape = (1, 3, input_height, input_width) # NCHW

image_input = ct.ImageType(
    name="input",
    shape=input_shape,
    color_layout=ct.colorlayout.RGB,
    scale=scale,
    bias=bias
)

In [ ]:
mlmodel_neuralnetwork = ct.convert(
    traced_model,
    inputs=[image_input],
    convert_to="neuralnetwork"
    # convert_to="mlprogram",
    # minimum_deployment_target=ct.target.iOS16,
    # compute_precision=ct.precision.FLOAT16
)

In [ ]:
# Set metadata
mlmodel_neuralnetwork.short_description = 'Wall segmentation model'
mlmodel_neuralnetwork.input_description['input'] = 'Input RGB image'

output = mlmodel_neuralnetwork._spec.description.output[0]
output.type.multiArrayType.shape.append(input_height)
output.type.multiArrayType.shape.append(input_width)

ct.utils.rename_feature(
    mlmodel_neuralnetwork._spec, 
    mlmodel_neuralnetwork.get_spec().description.output[0].name, 
    "output", 
    rename_inputs=True
)

mlmodel_neuralnetwork.output_description['output'] = "Wall segmentation map"

In [ ]:
model_filename = f'DeepLabV3Plus-mobileone_s1.neuralnetwork.mlmodel'
mlmodel_neuralnetwork.save(model_filename)
print(f'MLModel saved to {model_filename}')

In [ ]:
mlmodel_mlprogram = ct.convert(
    traced_model,
    inputs=[image_input],
    convert_to="mlprogram"
    # minimum_deployment_target=ct.target.iOS16,
    # compute_precision=ct.precision.FLOAT16
)

In [ ]:
spec = mlmodel_mlprogram.get_spec()
print("model type: {}".format(spec.WhichOneof('Type')))

In [ ]:
# Set metadata
mlmodel_mlprogram.short_description = 'Wall segmentation model'
mlmodel_mlprogram.input_description['input'] = 'Input RGB image'

# output = mlmodel_mlprogram._spec.description.output[0]
# output.type.multiArrayType.shape.append(input_height)
# output.type.multiArrayType.shape.append(input_width)

# ct.utils.rename_feature(
#     mlmodel_mlprogram._spec, 
#     mlmodel_mlprogram.get_spec().description.output[0].name, 
#     "output", 
#     rename_inputs=True
# )

# mlmodel_mlprogram.output_description['output'] = "Wall segmentation map"

labels_json = {"labels": ["background", "wall"]}

mlmodel_mlprogram.user_defined_metadata["com.apple.coreml.model.preview.type"] = "imageSegmenter"
mlmodel_mlprogram.user_defined_metadata['com.apple.coreml.model.preview.params'] = json.dumps(labels_json)



In [ ]:
model_filename = f'DeepLabV3Plus-mobileone_s1.mlprogram.mlpackage'
mlmodel_mlprogram.save(model_filename)
print(f'MLModel saved to {model_filename}')